In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Set NPM dan seed untuk reproduksibilitas
NPM = 19102
np.random.seed(NPM)

# Baca dan perbaiki data makanan
makan = pd.read_csv("makanan.csv")

# Perbaiki data yang missing di kolom harga
makan.loc[9, 'harga'] = 5000.0   # mi instan
makan.loc[10, 'harga'] = 3000.0  # roti
makan.loc[11, 'harga'] = 8000.0  # nasi telur

print("=== DATA MAKANAN ===")
print(makan)

# Parameter simulasi
budget_awal = 1e6  # rupiah
wkenyang_base = 0.8
wrasa_base = 0.7
wsehat_base = 0.9
wbosan_base = 0.2

SAMPLE_NUM = 500
SISA_MINIMUM = 150e3
SISA_MAKSIMUM = 300e3

# Inisialisasi array sesuai kode asli
sampleid = np.arange(0, SAMPLE_NUM)
skor = np.zeros(SAMPLE_NUM, dtype=np.double)
sisa = np.zeros(SAMPLE_NUM, dtype=np.double)
sehat = np.zeros(SAMPLE_NUM, dtype=np.double)
pilihankum = np.zeros(SAMPLE_NUM, dtype=np.double)

bin_sisa = np.arange(SISA_MINIMUM, SISA_MAKSIMUM, 1000)
resp_sehat = np.zeros((SAMPLE_NUM, len(bin_sisa)), dtype=np.double)
resp_pilih = np.zeros((SAMPLE_NUM, 61), dtype=np.int32)

print(f"\n=== SIMULASI {SAMPLE_NUM} SAMPEL ===")

# Simulasi sesuai struktur kode asli
for n in range(0, SAMPLE_NUM):
    budget = budget_awal
    skor_kumulatif = 0
    skor_sehat = 0

    for i in range(0, 31):  # 31 hari
        # makan pertama (pagi)
        pilihan = np.random.randint(0, len(makan))

        w_kenyang = np.random.normal(loc=wkenyang_base, scale=0.2)
        w_rasa = np.random.normal(loc=wrasa_base, scale=0.2)
        w_sehat = np.random.normal(loc=wsehat_base, scale=0.4)
        w_bosan = np.random.normal(loc=wbosan_base, scale=0.2)

        skor_kumulatif += (w_kenyang * makan.iloc[pilihan]['kenyang'] +
                          w_rasa * makan.iloc[pilihan]['rasa'] +
                          w_sehat * makan.iloc[pilihan]['sehat'] -
                          w_bosan * makan.iloc[pilihan]['bosan'])

        skor_sehat += makan.iloc[pilihan]['sehat'] - 0.5
        budget -= makan.iloc[pilihan]['harga']

        if 2*i < 61:
            resp_pilih[n][2*i] = pilihan

        # makan kedua (malam)
        pilihan = np.random.randint(0, len(makan))

        w_kenyang = np.random.normal(loc=wkenyang_base, scale=0.2)
        w_rasa = np.random.normal(loc=wrasa_base, scale=0.2)
        w_sehat = np.random.normal(loc=wsehat_base + 0.2, scale=0.4)
        w_bosan = np.random.normal(loc=wbosan_base * 2, scale=0.2)

        skor_kumulatif += (w_kenyang * makan.iloc[pilihan]['kenyang'] +
                          w_rasa * makan.iloc[pilihan]['rasa'] +
                          w_sehat * makan.iloc[pilihan]['sehat'] -
                          w_bosan * makan.iloc[pilihan]['bosan'])

        skor_sehat += makan.iloc[pilihan]['sehat'] - 0.5
        budget -= makan.iloc[pilihan]['harga']

        if 2*i + 1 < 61:
            resp_pilih[n][2*i + 1] = pilihan

    # Simpan hasil
    sisa[n] = budget
    skor[n] = skor_kumulatif
    sehat[n] = skor_sehat

    # Update resp_sehat berdasarkan bin sisa uang
    for j, bin_val in enumerate(bin_sisa):
        if bin_val <= budget < bin_val + 1000:
            resp_sehat[n][j] = skor_sehat
            break

print("\nSOAL 1: POLA MAKAN DENGAN NILAI KUMULATIF KESEHATAN (resp_sehat) POSITIF")
print("-" * 75)

sehat_positif_mask = sehat > 0
sampel_sehat_positif = np.where(sehat_positif_mask)[0]

print(f"Jumlah sampel dengan nilai kumulatif kesehatan positif: {len(sampel_sehat_positif)} dari {SAMPLE_NUM}")
print(f"Persentase: {len(sampel_sehat_positif)/SAMPLE_NUM*100:.1f}%")
print(f"Nilai kumulatif kesehatan positif:")
print(f"  - Minimum: {np.min(sehat[sampel_sehat_positif]):.3f}")
print(f"  - Maksimum: {np.max(sehat[sampel_sehat_positif]):.3f}")
print(f"  - Rata-rata: {np.mean(sehat[sampel_sehat_positif]):.3f}")

resp_sehat_positif = []
for n in range(SAMPLE_NUM):
    for j in range(len(bin_sisa)):
        if resp_sehat[n][j] > 0:
            resp_sehat_positif.append((n, j, resp_sehat[n][j]))

print(f"Total entri resp_sehat positif: {len(resp_sehat_positif)}")
print(f"ID Sampel dengan kesehatan positif: {sampel_sehat_positif[:15]}...")

print(f"\nSOAL 2: TITIK DENGAN SISA UANG POSITIF DARI SAMPEL SEHAT POSITIF")
print("-" * 75)

sisa_positif_dari_sehat = sampel_sehat_positif[sisa[sampel_sehat_positif] > 0]
print(f"Dari {len(sampel_sehat_positif)} sampel dengan kesehatan positif,")
print(f"yang memiliki sisa uang positif: {len(sisa_positif_dari_sehat)} sampel")
print(f"Statistik sisa uang:")
print(f"  - Minimum: Rp {np.min(sisa[sisa_positif_dari_sehat]):.0f}")
print(f"  - Maksimum: Rp {np.max(sisa[sisa_positif_dari_sehat]):.0f}")
print(f"  - Rata-rata: Rp {np.mean(sisa[sisa_positif_dari_sehat]):.0f}")

print(f"\nSOAL 3: SAMPEL DENGAN SKOR KUMULATIF TERBESAR (KESEHATAN POSITIF)")
print("-" * 75)

idx_skor_terbesar = sampel_sehat_positif[np.argmax(skor[sampel_sehat_positif])]
print(f"Sampel dengan skor kumulatif terbesar:")
print(f"  - Sampel ID: {idx_skor_terbesar}")
print(f"  - Skor kumulatif: {skor[idx_skor_terbesar]:.3f}")
print(f"  - Nilai kumulatif kesehatan: {sehat[idx_skor_terbesar]:.3f}")
print(f"  - Sisa uang: Rp {sisa[idx_skor_terbesar]:.0f}")

print(f"\nSOAL 4: JADWAL MAKANAN SAMPEL TERBAIK (ID: {idx_skor_terbesar})")
print("-" * 75)

print("JADWAL MAKAN 31 HARI:")
print("Hari | Pagi (Harga)          | Malam (Harga)         | Total Harian")
print("-" * 75)

total_bulanan = 0
for hari in range(31):
    if 2*hari < 61 and 2*hari + 1 < 61:
        pilihan_pagi = int(resp_pilih[idx_skor_terbesar][2*hari])
        pilihan_malam = int(resp_pilih[idx_skor_terbesar][2*hari + 1])

        makanan_pagi = makan.iloc[pilihan_pagi]['makanan']
        makanan_malam = makan.iloc[pilihan_malam]['makanan']
        harga_pagi = makan.iloc[pilihan_pagi]['harga']
        harga_malam = makan.iloc[pilihan_malam]['harga']

        total_harian = harga_pagi + harga_malam
        total_bulanan += total_harian

        print(f"{hari+1:2d}   | {makanan_pagi:12} ({harga_pagi:5.0f}) | {makanan_malam:12} ({harga_malam:5.0f}) | {total_harian:6.0f}")

print("-" * 75)
print(f"TOTAL PENGELUARAN MAKAN 31 HARI: Rp {total_bulanan:.0f}")
print(f"SISA UANG DARI BUDGET Rp 1.000.000: Rp {budget_awal - total_bulanan:.0f}")

pilihan_flat = resp_pilih[idx_skor_terbesar][resp_pilih[idx_skor_terbesar] < len(makan)]
pilihan_unik, counts = np.unique(pilihan_flat, return_counts=True)

print(f"\nANALISIS POLA MAKANAN SAMPEL TERBAIK:")
print("Ranking | Makanan        | Frekuensi | Persentase")
print("-" * 50)
makanan_ranking = [(int(pilihan), count) for pilihan, count in zip(pilihan_unik, counts)]
makanan_ranking.sort(key=lambda x: x[1], reverse=True)

for i, (idx_makanan, frekuensi) in enumerate(makanan_ranking):
    nama_makanan = makan.iloc[idx_makanan]['makanan']
    persen = frekuensi / len(pilihan_flat) * 100
    print(f"{i+1:2d}      | {nama_makanan:12} | {frekuensi:8d}  | {persen:6.1f}%")

=== DATA MAKANAN ===
       makanan  kenyang  sehat  rasa  bosan    harga
0   ayamgeprek      0.9    0.5   0.7    0.2  15000.0
1   nasipadang      1.0    0.8   0.9    0.5  17000.0
2      mieayam      1.0    0.6   0.9    0.3  10000.0
3        bakso      0.7    0.3   1.0    0.3  10000.0
4       cireng      0.5    0.4   0.8    0.7  10000.0
5   nasigoreng      0.8    0.7   1.0    0.5  14000.0
6       seblak      0.6    0.2   0.8    0.8  15000.0
7     gorengan      0.5    0.2   0.5    0.5  10000.0
8       capcay      0.6    0.9   0.8    0.4  15000.0
9   mi instan       0.5    0.1   0.7    0.2   5000.0
10        roti      0.3    0.3   0.9    0.3   3000.0
11  nasi telur      0.7    0.4   0.8    0.8   8000.0
12      warteg      0.8    0.8   0.6    0.2  18000.0

=== SIMULASI 500 SAMPEL ===

SOAL 1: POLA MAKAN DENGAN NILAI KUMULATIF KESEHATAN (resp_sehat) POSITIF
---------------------------------------------------------------------------
Jumlah sampel dengan nilai kumulatif kesehatan positif: 11